In [1]:
def IncSerial():
    global serial
    serial = serial +1

In [2]:
import rsa
import base64
import hashlib
def encrypt(PT,publicKey):
    cipher = rsa.encrypt(PT.encode(), publicKey)
    base64Text = base64.b64encode(cipher).decode()
    return base64Text

def decrypt(CT, privateKey):
    plain = rsa.decrypt(base64.b64decode(CT.encode()), privateKey)
    return plain

# def Hash(txt):
#     hashfn = hashlib.sha256()
#     if len(txt)>1:
#         for i in txt:
#             hashfn.update(i.encode())
#         return hashfn.hexdigest()
#     hashfn.update(txt.encode())
#     return hashfn.hexdigest()

def sign(PT, privateKey):
    PT = PT.encode()
    PT= rsa.compute_hash(PT, 'SHA-256')
#     print(PT.hex())
    signature = rsa.sign_hash(PT, privateKey, 'SHA-256')
    return signature
    
def verifySign(ST,signature,publicKey):
    ST= ST.encode()
    try:
        rsa.verify(ST, signature, publicKey)
    except rsa.pkcs1.VerificationError:
        print('--------------Verification failed\n')
        return 0
    else:
        print('--------------Verification succeed\n')
        return 1
        
def KeyGen():
    publicKey, privateKey = rsa.newkeys(512)
    return publicKey, privateKey
CAPubKey=''
CAPriKey=''
def verifyCertificate(Obj,CAP):
    if str(type(Obj)) == "<class '__main__.certificate'>" or str(type(Obj)) == "<class 'CA.certificate'>" :
        now = datetime.utcnow()
        global CAPubKey
        if Obj.signature != '' or Obj.periodOfValidityStart < now or Obj.periodOfValidityStop > now :
            txt = Obj.CertificateData()
            return verifySign(txt,Obj.signature,CAP)
def genCAKeys():
    global CAPriKey , CAPubKey
    if CAPriKey == '':
        CAPubKey, CAPriKey = KeyGen()

def genCertificate(subName, subPubKey, role):
    a = certificate(subName, subPubKey, role)
    return a.transmitCertificate()
# a=KeyGen()

# txt='Hello World'
# b = sign(txt,a[1])
# rsa.verify(txt.encode(),b,a[0])
# def GenerateCertificate(subName, SubPublicKey):
# verifySign('Hello World',b , a[0])
# verifySign('Hello World',sign('Hello World',a[1] ),a[0])

In [3]:
from datetime import datetime, timedelta
genCAKeys()
serial = 101
class certificate :
    version_No = 'x.509'
    serial_No = 0
    signatureAlgorithmID= 'SHA-256'
    issuerName = 'CA'
    periodOfValidityStart = ''
    periodOfValidityStop = ''
    subjectName=''
    subjectPublicKey = ''
    extension=0
    signature= ''
    def __init__(self,subName='',subPubKey='',role=0 ):
        if subName == '' or subPubKey == '' or role == 0 :
            now = datetime.utcnow()
            self.periodOfValidityStop = now - timedelta(days=1*365)       
        else :   
            global CAPriKey , CAPubKey, serial
            self.serial_No = serial
            IncSerial()
            now = datetime.utcnow()
            self.periodOfValidityStart= now
            self.periodOfValidityStop = now + timedelta(days=1*365)
            self.subjectName = subName
            self.subjectPublicKey = subPubKey
            self.extension = role
            self.signature = sign(self.CertificateData(),CAPriKey)
    def CertificateData(self):
        txt= self.version_No + str(self.serial_No) + self.signatureAlgorithmID +\
        self.issuerName + str(self.periodOfValidityStart) + str(self.periodOfValidityStop) +\
        self.subjectName+str(self.subjectPublicKey)+str(self.extension)
        return txt
    def transmitCertificate(self):
        cert = {
            "v_no" : self.version_No,
            "s_no" : self.serial_No,
            "sa_id" : self.signatureAlgorithmID,
            "i_n" : self.issuerName,
            "v_start" : self.periodOfValidityStart,
            "v_stop" : self.periodOfValidityStop,
            "s_n" : self.subjectName,
            "s_PK" : self.subjectPublicKey,
            "ext" : self.extension,
            "sig" : self.signature
        }
        return cert
    def dict2cert(self,certData):
        self.version_No = certData["v_no"]
        self.serial_No = certData["s_no"]
        self.signatureAlgorithmID = certData["sa_id"]
        self.issuerName = certData["i_n"]
        self.periodOfValidityStart= certData["v_start"]
        self.periodOfValidityStop = certData["v_stop"]
        self.subjectName = certData["s_n"]
        self.subjectPublicKey = certData["s_PK"]
        self.extension = certData["ext"]
        self.signature = certData["sig"]
    
        

In [6]:
# from datetime import datetime, timedelta
# import ipaddress

# def generate_selfsigned_cert(hostname, ip_addresses=None, key=None):
#     """Generates self signed certificate for a hostname, and optional IP addresses."""
#     from cryptography import x509
#     from cryptography.x509.oid import NameOID
#     from cryptography.hazmat.primitives import hashes
#     from cryptography.hazmat.backends import default_backend
#     from cryptography.hazmat.primitives import serialization
#     from cryptography.hazmat.primitives.asymmetric import rsa
    
#     # Generate our key
#     if key is None:
#         key = rsa.generate_private_key(
#             public_exponent=65537,
#             key_size=2048,
#             backend=default_backend(),
#         )
    
#     name = x509.Name([
#         x509.NameAttribute(NameOID.COMMON_NAME, hostname)
#     ])
 
#     # best practice seem to be to include the hostname in the SAN, which *SHOULD* mean COMMON_NAME is ignored.    
#     alt_names = [x509.DNSName(hostname)]
    
#     # allow addressing by IP, for when you don't have real DNS (common in most testing scenarios 
#     if ip_addresses:
#         for addr in ip_addresses:
#             # openssl wants DNSnames for ips...
#             alt_names.append(x509.DNSName(addr))
#             # ... whereas golang's crypto/tls is stricter, and needs IPAddresses
#             # note: older versions of cryptography do not understand ip_address objects
#             alt_names.append(x509.IPAddress(ipaddress.ip_address(addr)))
    
#     san = x509.SubjectAlternativeName(alt_names)
    
#     # path_len=0 means this cert can only sign itself, not other certs.
#     basic_contraints = x509.BasicConstraints(ca=True, path_length=0)
#     now = datetime.utcnow()
#     cert = (
#         x509.CertificateBuilder()
#         .subject_name(name)
#         .issuer_name(name)
#         .public_key(key.public_key())
#         .serial_number(1000)
#         .not_valid_before(now)
#         .not_valid_after(now + timedelta(days=10*365))
#         .add_extension(basic_contraints, False)
#         .add_extension(san, False)
#         .sign(key, hashes.SHA256(), default_backend())
#     )
#     cert_pem = cert.public_bytes(encoding=serialization.Encoding.PEM)
#     key_pem = key.private_bytes(
#         encoding=serialization.Encoding.PEM,
#         format=serialization.PrivateFormat.TraditionalOpenSSL,
#         encryption_algorithm=serialization.NoEncryption(),
#     )

#     return cert_pem, key_pem, cert


In [4]:
# b,c = KeyGen()
# C1 = certificate('ECU1',b,2)
# txt = C1.CertificateData()
# genCertificate('ECU1',b,2)
# verifySign(txt,C1.Signature,CAPriKey)
# t = str(C1.Signature.hex())
# t=t+'03'
# t =base64.b64decode(t.encode())
# print(txt)
# print(sign(txt,CAPriKey))
# print(C1.signature)
# verifySign(txt,C1.signature,CAPriKey)
# verifyCertificate(C1)

In [5]:
# import socket
# import pickle
# client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# client.connect(('127.0.0.1', 8080))
# closeCon = 0
# data = pickle.dumps(C1.transmitCertificate())
# client.send(data)
# client.close()


In [6]:
# import socket
# import pickle
# serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# serv.bind(('127.0.0.7', 8080))
# serv.listen(5)
# a=0
# b =0
# while a==0:
#   conn, addr = serv.accept()
#   from_client = ''
#   while b==0:
#     data = conn.recv(4096)
#     if not data: break
# #     from_client = data.decode('utf8')
#     from_client = pickle.loads(data)
#     if from_client['req'] == 1:
#         data = pickle.dumps(CAPubKey)
#         conn.send(data)
#     a,b=1,1
    
#   conn.close()